In [1]:
import networkx as nx
import pickle
import matplotlib.pyplot as plt

In [2]:
#Load the Dataset
data = pickle.load(open('labeled_data\\fully_connected_network.pkl', 'rb'))

In [22]:
# Creating the structure of the tasks
project = nx.Graph()
node_list = ["java", "cpp", "web"]
edge_list = [("java", "web"),("cpp", "web")]
project.add_nodes_from(node_list)
project.add_edges_from(edge_list)

In [18]:
for u in project.nodes:
    if u == 'java':
        print("Java")
    else:
        print("Not Java")

Java
Not Java
Not Java


In [4]:
# Initialize Sets for each Team
Team_java = set()
Team_cpp = set()
Team_web = set()

Task_Java_skills = set(['Mobile App Development', 'Android', 'iPhone', 'Java',
                       'Linux', 'System Admin', 'Flutter', 'HTML', 'Pine Script', 'eCommerce', 'Swift',
                        'Web Scraping', 'Flutter', 'ASP.NET', 'MySQL', 'API', 'Amazon Web Services',
                        'Android Studio', 'API Integration'])
Task_Cpp_skills = set(['C++ Programming', 'C Programming', 'C# Programming',
                      'Visual Basic', 'Software Architecture', 'PHP', 'Linux', 'JavaScript',
                       'Blockchain', 'API', 'Engineering', 'MySQL', '.NET', 'API Integration'])
Task_Web_skills = set(['SEO', 'Website Design', 'PHP', 'WordPress', 'HTML', 'Website Design',
                      'JavaScript', 'HTML5', 'PHP', 'JavaScript', 'React.js', 'Graphic Design',
                       'CSS', 'Logo Design', 'Photoshop', 'Illustrator', 'eCommerce', 'Prestashop',
                       'PSD to HTML''Content Marketing', 'API', 'Python', 'Web Hosting', 'PSD to HTML',
                       'Advertising', 'Shopify', 'App Developer', 'Django'])

Covered_Java_skills = set()
Covered_Cpp_skills = set()
Covered_Web_skills = set()

In [5]:
# Create teams
# Define the attribute to filter by
attribute_to_filter = 'label'
java_value = 'java'
cpp_value = 'cpp'
web_value = 'web'

# Filter nodes based on the attribute
filtered_java_nodes = [node for node, data in data.nodes(data=True) if data.get(attribute_to_filter) == java_value]
filtered_cpp_nodes = [node for node, data in data.nodes(data=True) if data.get(attribute_to_filter) == cpp_value]
filtered_web_nodes = [node for node, data in data.nodes(data=True) if data.get(attribute_to_filter) == web_value]

# Create a subgraph with the filtered nodes
java_network = data.subgraph(filtered_java_nodes)
cpp_network = data.subgraph(filtered_cpp_nodes)
web_network = data.subgraph(filtered_web_nodes)

In [6]:
java_network.nodes()

NodeView((651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 

In [7]:
def greedyCover(network, skills):
    covered_skills = set()
    network_copy = network.copy()
    selected_set = set()

    while len(skills-covered_skills) > 0:
        max_cover = set()
        top_node = None
        for node in network_copy.nodes:
            local_skills = set(network_copy.nodes[node]['skills'])
            # print(f"Node Skills --> {local_skills}")
            if len(local_skills.intersection(skills-covered_skills)) > len(max_cover):
                max_cover = local_skills
                top_node = node
        # print(f"Best Skill set --> {max_cover}")
        if top_node is not None:
            selected_set.add(top_node)
            covered_skills = covered_skills.union(max_cover)
            network_copy.remove_node(top_node)
        else:
            break
        # print(f"Covered Skills = {covered_skills}")
    return list(selected_set)

In [8]:
cpp_best = greedyCover(cpp_network, Task_Cpp_skills)

In [9]:
java_best = greedyCover(java_network, Task_Java_skills)

In [10]:
web_best = greedyCover(web_network, Task_Web_skills)

In [ ]:
best = []
best.extend(cpp_best)
best.extend(java_best)
best.extend(web_best)
best

In [12]:
best_in_network = data.subgraph(best)

In [25]:
java_best_network = data.subgraph(java_best)
cpp_best_network = data.subgraph(cpp_best)
web_best_network = data.subgraph(web_best)

In [13]:
best_in_network.nodes[1]

{'location': 'Venezuela',
 'skills': ['Metatrader',
  'C++ Programming',
  'C Programming',
  'C# Programming',
  'Visual Basic'],
 'label': 'cpp'}

In [26]:
def distance_btn_nodes(G, source_node, target_node):
    distance = nx.shortest_path_length(G, source=source_node, target=target_node, weight="weight")
    return distance

def distance_btn_node_and_team(G, source_node, target_team):
    distance_to_team = 0
    for node in target_team.nodes:
        distance_to_team = distance_to_team + distance_btn_nodes(G, source_node, node)
    return distance_to_team

def coordinator(G, source_team, target_team):
    coordinator = None
    min_distance = float('Inf') # Set the minimum distance to a very large number
    for node in source_team:
        node_distance = distance_btn_node_and_team(G, node, target_team)
        if node_distance < min_distance:
            min_distance = node_distance
            coordinator = node
    print(f"Coordinator Distance = {min_distance}")
    return coordinator

In [27]:
def find_coordinators(G, P):
    coordinators = set()
    
    for (u, v) in P.edges:
        new_coordinator = None
        if u == 'java':
            source_team = java_best_network
        elif u == 'cpp':
            source_team = cpp_best_network
        else:
            source_team = web_best

        if v == 'java':
            target_team = java_best_network
        elif v == 'cpp':
            target_team = cpp_best_network
        else:
            target_team = web_best_network

        new_coordinator = coordinator(G, source_team, target_team)
        coordinators.add(new_coordinator)
    return coordinators

In [28]:
find_coordinators(data, project)

Coordinator Distance = 0.8023172670460694
Coordinator Distance = 1.0905277443752528


{38, 662}

In [29]:
data.nodes[38]

{'location': 'Pakistan',
 'skills': ['Software Architecture',
  'PHP',
  'C++ Programming',
  'MySQL',
  'C Programming'],
 'label': 'cpp'}

In [30]:
data.nodes[662]

{'location': 'India',
 'skills': ['HTML', 'PHP', 'Sharepoint', 'Website Design', 'eCommerce'],
 'label': 'java'}

In [31]:
merged_graph = nx.compose(cpp_best_network, java_best_network)
coordinator(data, web_best_network, merged_graph)

Coordinator Distance = 1.8489753253435084


1433

In [32]:
web_to_cpp_coordinator = coordinator(data, web_best_network, cpp_best_network)
print(f"Coordinator from Web to Cpp team is : {web_to_cpp_coordinator}")
web_to_java_coordinator = coordinator(data, web_best_network, java_best_network)
print(f"\nCoordinator from Web to Cpp team is : {web_to_java_coordinator}")